In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten
import tensorflow as tf
import numpy as np
import os
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder

from bayes_opt import BayesianOptimization

In [ ]:
train = pd.read_csv('./train_data.csv', encoding='cp949')
test = pd.read_csv('./test_data.csv', encoding='cp949')

/content/drive/MyDrive/Colab Notebooks/스마트빌딩


## 전처리

In [ ]:
#결측치 확인
def shownull(df):
    global missval_name
    print(f"데이터셋 차원 확인 \n {df.shape}")
    missval = df.isnull().sum()[df.isnull().sum()!=0]
    print(f"결측값 있는 변수 확인 \n {missval}")
    missval_name = missval.index # 결측치가 존재하는 열이 저장된 변수

#이상치 확인
def zscore_out(df, threshold = 3):
    mean = np.mean(df)
    std = np.std(df)
    z_score = np.abs((df - df.mean()) / df.std())
    outliers = np.where(z_score > threshold)
    # outlier_idx = outliers.nonzero()[0]
    print(outliers[0])

#시드고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(84)

### 결측치 대체

In [ ]:
# Train set에 대해서 시간 변수 결측치 대체

train['regdate'] = pd.to_datetime(train['regdate'])

# 시작일과 종료일 설정
start_date = train['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = train['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(train)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(train.loc[i, 'regdate']):
        train.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

#제대로 작동됨

In [ ]:
# Test set에 대해서 시간 변수 결측치 대체

test['regdate'] = pd.to_datetime(test['regdate'])

# 시작일과 종료일 설정
start_date = test['regdate'].min()  # 시작일: 2023-01-01 00:00
end_date = test['regdate'].max()  # 종료일: 2023-01-27 23:59    -------> min, max를 이용해서 data leakage 문제를 해결

# 결측치를 수정하고 이전 시간으로부터 순서대로 채워넣음
current_date = start_date  # 현재 날짜 초기화

for i in range(len(test)):
    # 결측치인 경우 이전 시간으로 대체
    if pd.isnull(test.loc[i, 'regdate']):
        test.loc[i, 'regdate'] = current_date
    
    # 다음 시간으로 이동
    current_date += timedelta(minutes=1)
    
    # 종료일인 경우 시작일로 되돌아감
    if current_date > end_date:
        current_date = start_date

### 독립변수 추가

In [ ]:
# TRAIN 요일 변수 추가
daysofweek = []
for i in range(len(train)) :
    dayweek = train.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

train['dayofweek'] = daysofweek



#월요일 0 ~ 일요일6

<!--  -->

In [ ]:
# TEST 요일 변수 추가

daysofweek = []
for i in range(len(test)) :
    dayweek = test.loc[i, 'regdate'].dayofweek
    daysofweek.append(dayweek)

test['dayofweek'] = daysofweek

In [ ]:
# TRAIN 시간 -> 10진법 변환
train['time'] = train['regdate'].dt.strftime('%H:%M').str.split().str[0]
train['time'] = train['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [ ]:
# TEST 시간 -> 10진법 변환
test['time'] = test['regdate'].dt.strftime('%H:%M').str.split().str[0]
test['time'] = test['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

In [ ]:
it_is_not_na = np.where(~train[['PIR']].isnull())[0]

train.loc[it_is_not_na,['PIR']] = train.loc[it_is_not_na,['PIR']].astype(np.int64)

### 결측치 대체

In [ ]:
# def cat_num_na(df) :

#     #변수 타입별로 분리
#     cat_na = df.select_dtypes(include=['object', 'int64'])
#     num_na = df.select_dtypes(include=['float64'])

#     #범주형 변수 최빈값으로 변환
#     for col in cat_na:
#         df[col].fillna(df[col].mode()[0], inplace=True)

#     #수치형 변수 중위값 대체
#     for col2 in num_na:
#         df[col2].fillna(df[col2].median(), inplace=True)

In [ ]:
def make_area(df) :
    # 입력된 날짜 변수의 차이 계산
    date_range = df['regdate'].max() - df['regdate'].min()

    # 간격당 행의 개수 계산
    lowsbydate = ((date_range.days + 1) * 1440) #하루 1440분
    group_num = len(train) / lowsbydate

    for i in range(int(group_num)):
        start_index = i * lowsbydate
        end_index = (i + 1) * lowsbydate
        df.loc[start_index : end_index, 'area'] = i + 1

def inter_by_area(df):
    df.update(df.groupby('area').transform(lambda x: x.interpolate(method = 'pad')))
    return df

In [ ]:
shownull(train_nona)

데이터셋 차원 확인 
 (388800, 15)
결측값 있는 변수 확인 
 Series([], dtype: int64)


### 파일 분할 및 스케일링

In [ ]:
# #One-Hot Encoding
# ohe = OneHotEncoder(sparse=False)
# onehot_train = ohe.fit_transform(train_nona[['dayofweek']])
# onehot_test = ohe.transform(test_nona[['dayofweek']])

# onehot_train2 = pd.DataFrame(onehot_train, columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
# onehot_test2 = pd.DataFrame(onehot_test, columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# onehot_train3 = pd.concat([train_nona.drop('dayofweek', axis=1), onehot_train2], axis=1)
# onehot_test3 = pd.concat([test_nona.drop('dayofweek', axis=1), onehot_test2], axis=1)

In [ ]:
X_train = train.drop('재실인원', axis=1)
y_train = train['재실인원']

X_test = test.drop('재실인원', axis=1)
y_test = test['재실인원']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3)

In [ ]:
#regdate 제거한 뒤 확인

X_train = X_train.drop(['regdate','area','dayofweek'], axis=1)
X_valid = X_valid.drop(['regdate','area','dayofweek'], axis=1)
X_test = X_test.drop(['regdate','area','dayofweek'], axis=1)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [ ]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10
0,0.519084,0.223684,0.165307,0.040580,0.027451,0.014120,0.057554,0.001038,0.013934,0.0,0.748436
1,0.381679,0.223684,0.153285,0.056522,0.038235,0.019667,0.007194,0.000961,0.012883,0.0,0.232801
2,0.381679,0.157895,0.153714,0.011594,0.007843,0.004034,0.007194,0.000275,0.003831,0.0,0.045865
3,0.221374,0.171053,0.156290,0.004348,0.002941,0.001513,0.028777,0.000046,0.000618,0.0,0.670605
4,0.442748,0.276316,0.313010,0.050725,0.034314,0.018154,0.007194,0.001663,0.022306,0.0,0.685198
...,...,...,...,...,...,...,...,...,...,...,...
272155,0.519084,0.236842,0.320739,0.002899,0.001961,0.001009,0.064748,0.000931,0.012543,0.0,0.760250
272156,0.725191,0.342105,0.297982,0.013043,0.008824,0.004539,0.043165,0.000839,0.011277,0.0,0.759555
272157,0.541985,0.223684,0.122370,0.004348,0.002941,0.001513,0.007194,0.000061,0.000989,0.0,0.243224
272158,0.732824,0.407895,0.468871,0.042029,0.028431,0.014624,0.007194,0.001953,0.026013,0.0,0.904795


In [ ]:
train_nona.shape

(388800, 15)

In [ ]:
X_train = np.reshape(X_train, (len(X_train),1,11)).astype(float)
X_valid = np.reshape(X_valid, (len(X_valid),1,11)).astype(float)
X_test = np.reshape(X_test, (len(X_test),1,11)).astype(float)

In [ ]:
X_train.shape

(272160, 1, 11)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,LSTM
from keras.layers import Input, Dense, Activation, Flatten, Dropout
# # GRU 모형 적용
# model4 = Sequential()
# model4.add(GRU(32, input_shape=(1,12)))
# model4.add(Dense(1))
# model4.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
# model4.fit(X_train, y_train, epochs=3, batch_size=500)

In [ ]:
mse =[]
def buildModel(neurons,drop,nb_batch,hidden_layers):  
    nb_epoch=5
    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = tf.keras.Input(shape=(1,11))

    x = tf.keras.layers.LSTM(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=0.2, activation='relu')(inputs)  
    # x = tf.keras.layers.Dense(neurons*(2**(hidden_layers+1)), activation='relu')(inputs)

    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = tf.keras.layers.LSTM(neurons * (2 ** (hidden_layers + 1)), return_sequences=True)(x)
    x = tf.keras.layers.LSTM(neurons * (2 ** (hidden_layers + 1)))(x)
    x = tf.keras.layers.Dropout(drop)(x)

    # output
    predictions = Dense(1)(x)


    # modeling
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



    # learning
    model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


    # evaluate the model with test data
    loss, mse =model.evaluate(X_test, y_test)
    
    if mse < 0.85: 
        model.save(f"./model/rmse/{mse}_lstm.h5")
        print("모델이 저장 되었습니다.")
        print("mse : ",mse)
    
    return mse

In [ ]:
pbounds = {
        # define the range of each parameter
        'drop' : (0.5,0.9),
        "neurons" : (12,24),
        'nb_batch' : (512,1024),
        'hidden_layers' : (0,3)}

In [ ]:
bo=BayesianOptimization(f=buildModel, pbounds=pbounds, verbose=2, random_state=42 )

In [ ]:
bo.maximize(
   init_points=2,
   n_iter= 100
)

|   iter    |  target   |   drop    | hidden... | nb_batch  |  neurons  |
-------------------------------------------------------------------------
Epoch 1/5
308/308 [==============================] - 19s 23ms/step - loss: 1.3862 - mean_squared_error: 1.3862 - val_loss: 0.8404 - val_mean_squared_error: 0.8404
Epoch 2/5
308/308 [==============================] - 5s 17ms/step - loss: 0.8320 - mean_squared_error: 0.8320 - val_loss: 0.7589 - val_mean_squared_error: 0.7589
Epoch 3/5
308/308 [==============================] - 5s 15ms/step - loss: 0.7729 - mean_squared_error: 0.7729 - val_loss: 0.7146 - val_mean_squared_error: 0.7146
Epoch 4/5
308/308 [==============================] - 6s 20ms/step - loss: 0.7306 - mean_squared_error: 0.7306 - val_loss: 0.7340 - val_mean_squared_error: 0.7340
Epoch 5/5
2657/2657 [==============================] - 12s 5ms/step - loss: 1.4341 - mean_squared_error: 1.4341


| 1         | 1.434     | 0.6498    | 2.852     | 886.8     | 19.18     |
Epoch 1/5
504/504 [==============================] - 13s 16ms/step - loss: 1.6458 - mean_squared_error: 1.6458 - val_loss: 1.0935 - val_mean_squared_error: 1.0935
Epoch 2/5
504/504 [==============================] - 6s 12ms/step - loss: 1.0282 - mean_squared_error: 1.0282 - val_loss: 0.8665 - val_mean_squared_error: 0.8665
Epoch 3/5
504/504 [==============================] - 7s 15ms/step - loss: 0.9231 - mean_squared_error: 0.9231 - val_loss: 0.8290 - val_mean_squared_error: 0.8290
Epoch 4/5
504/504 [==============================] - 6s 11ms/step - loss: 0.8926 - mean_squared_error: 0.8926 - val_loss: 0.8098 - val_mean_squared_error: 0.8098
Epoch 5/5
2657/2657 [==============================] - 9s 3ms/step - loss: 1.3856 - mean_squared_error: 1.3856


| 2         | 1.386     | 0.5624    | 0.468     | 541.7     | 22.39     |
Epoch 1/5
307/307 [==============================] - 13s 22ms/step - loss: 1.6427 - mean_squared_error: 1.6427 - val_loss: 0.9891 - val_mean_squared_error: 0.9891
Epoch 2/5
307/307 [==============================] - 4s 12ms/step - loss: 0.9419 - mean_squared_error: 0.9419 - val_loss: 0.8376 - val_mean_squared_error: 0.8376
Epoch 3/5
307/307 [==============================] - 5s 16ms/step - loss: 0.8614 - mean_squared_error: 0.8614 - val_loss: 0.7793 - val_mean_squared_error: 0.7793
Epoch 4/5
307/307 [==============================] - 5s 15ms/step - loss: 0.8164 - mean_squared_error: 0.8164 - val_loss: 0.7499 - val_mean_squared_error: 0.7499
Epoch 5/5
2657/2657 [==============================] - 10s 4ms/step - loss: 0.9690 - mean_squared_error: 0.9690
| 3         | 0.969     | 0.5816    | 1.665     | 887.3     | 19.39     |


Epoch 1/5
333/333 [==============================] - 10s 14ms/step - loss: 1.8382 - mean_squared_error: 1.8382 - val_loss: 1.1723 - val_mean_squared_error: 1.1723
Epoch 2/5
333/333 [==============================] - 4s 12ms/step - loss: 1.1093 - mean_squared_error: 1.1093 - val_loss: 0.9157 - val_mean_squared_error: 0.9157
Epoch 3/5
333/333 [==============================] - 5s 15ms/step - loss: 0.9518 - mean_squared_error: 0.9518 - val_loss: 0.8544 - val_mean_squared_error: 0.8544
Epoch 4/5
333/333 [==============================] - 4s 12ms/step - loss: 0.9184 - mean_squared_error: 0.9184 - val_loss: 0.8356 - val_mean_squared_error: 0.8356
Epoch 5/5
2657/2657 [==============================] - 8s 3ms/step - loss: 23.9494 - mean_squared_error: 23.9494
| 4         | 23.95     | 0.5197    | 0.05545   | 819.5     | 20.69     |


Epoch 1/5
345/345 [==============================] - 19s 28ms/step - loss: 1.2839 - mean_squared_error: 1.2839 - val_loss: 0.8116 - val_mean_squared_error: 0.8116
Epoch 2/5
345/345 [==============================] - 6s 17ms/step - loss: 0.8585 - mean_squared_error: 0.8585 - val_loss: 0.7156 - val_mean_squared_error: 0.7156
Epoch 3/5
345/345 [==============================] - 7s 21ms/step - loss: 0.7832 - mean_squared_error: 0.7832 - val_loss: 0.6495 - val_mean_squared_error: 0.6495
Epoch 4/5
345/345 [==============================] - 6s 16ms/step - loss: 0.7236 - mean_squared_error: 0.7236 - val_loss: 0.6133 - val_mean_squared_error: 0.6133
Epoch 5/5
2657/2657 [==============================] - 13s 5ms/step - loss: 2.6771 - mean_squared_error: 2.6771
| 5         | 2.677     | 0.9       | 3.0       | 789.6     | 15.17     |


...

| 97        | 2.164     | 0.5293    | 2.926     | 860.1     | 19.22     |
Epoch 1/5
317/317 [==============================] - 15s 24ms/step - loss: 1.3074 - mean_squared_error: 1.3074 - val_loss: 0.8336 - val_mean_squared_error: 0.8336
Epoch 2/5
317/317 [==============================] - 5s 15ms/step - loss: 0.8030 - mean_squared_error: 0.8030 - val_loss: 0.7382 - val_mean_squared_error: 0.7382
Epoch 3/5
317/317 [==============================] - 6s 20ms/step - loss: 0.7421 - mean_squared_error: 0.7421 - val_loss: 0.7010 - val_mean_squared_error: 0.7010
Epoch 4/5
317/317 [==============================] - 5s 15ms/step - loss: 0.7025 - mean_squared_error: 0.7025 - val_loss: 0.7070 - val_mean_squared_error: 0.7070
Epoch 5/5
2657/2657 [==============================] - 12s 5ms/step - loss: 2.8055 - mean_squared_error: 2.8055
| 98        | 2.806     | 0.5212    | 2.086     | 861.1     | 23.91     |


Epoch 1/5
299/299 [==============================] - 13s 17ms/step - loss: 1.7432 - mean_squared_error: 1.7432 - val_loss: 1.1080 - val_mean_squared_error: 1.1080
Epoch 2/5
299/299 [==============================] - 5s 17ms/step - loss: 1.0114 - mean_squared_error: 1.0114 - val_loss: 0.8679 - val_mean_squared_error: 0.8679
Epoch 3/5
299/299 [==============================] - 5s 17ms/step - loss: 0.9248 - mean_squared_error: 0.9248 - val_loss: 0.8317 - val_mean_squared_error: 0.8317
Epoch 4/5
299/299 [==============================] - 4s 15ms/step - loss: 0.8910 - mean_squared_error: 0.8910 - val_loss: 0.8004 - val_mean_squared_error: 0.8004
Epoch 5/5
2657/2657 [==============================] - 10s 4ms/step - loss: 1.5382 - mean_squared_error: 1.5382


| 99        | 1.538     | 0.615     | 1.602     | 912.2     | 15.39     |
Epoch 1/5
269/269 [==============================] - 10s 16ms/step - loss: 1.8683 - mean_squared_error: 1.8683 - val_loss: 1.1207 - val_mean_squared_error: 1.1207
Epoch 2/5
269/269 [==============================] - 5s 17ms/step - loss: 1.1117 - mean_squared_error: 1.1117 - val_loss: 0.8682 - val_mean_squared_error: 0.8682
Epoch 3/5
269/269 [==============================] - 4s 16ms/step - loss: 0.9923 - mean_squared_error: 0.9923 - val_loss: 0.8160 - val_mean_squared_error: 0.8160
Epoch 4/5
269/269 [==============================] - 4s 14ms/step - loss: 0.9482 - mean_squared_error: 0.9482 - val_loss: 0.7992 - val_mean_squared_error: 0.7992
Epoch 5/5
2657/2657 [==============================] - 11s 4ms/step - loss: 1.1202 - mean_squared_error: 1.1202


| 100       | 1.12      | 0.798     | 1.565     | 1.015e+03 | 16.69     |
Epoch 1/5
333/333 [==============================] - 16s 24ms/step - loss: 1.4398 - mean_squared_error: 1.4398 - val_loss: 0.8436 - val_mean_squared_error: 0.8436
Epoch 2/5
333/333 [==============================] - 5s 16ms/step - loss: 0.9302 - mean_squared_error: 0.9302 - val_loss: 0.7608 - val_mean_squared_error: 0.7608
Epoch 3/5
333/333 [==============================] - 7s 21ms/step - loss: 0.8656 - mean_squared_error: 0.8656 - val_loss: 0.7287 - val_mean_squared_error: 0.7287
Epoch 4/5
333/333 [==============================] - 5s 15ms/step - loss: 0.8089 - mean_squared_error: 0.8089 - val_loss: 0.6745 - val_mean_squared_error: 0.6745
Epoch 5/5
2657/2657 [==============================] - 12s 5ms/step - loss: 3.6394 - mean_squared_error: 3.6394
| 101       | 3.639     | 0.8956    | 2.693     | 819.0     | 21.58     |


Epoch 1/5
517/517 [==============================] - 19s 19ms/step - loss: 1.2432 - mean_squared_error: 1.2432 - val_loss: 0.8109 - val_mean_squared_error: 0.8109
Epoch 2/5
517/517 [==============================] - 9s 18ms/step - loss: 0.8687 - mean_squared_error: 0.8687 - val_loss: 0.7334 - val_mean_squared_error: 0.7334
Epoch 3/5
517/517 [==============================] - 9s 18ms/step - loss: 0.7971 - mean_squared_error: 0.7971 - val_loss: 0.6688 - val_mean_squared_error: 0.6688
Epoch 4/5
517/517 [==============================] - 9s 17ms/step - loss: 0.7419 - mean_squared_error: 0.7419 - val_loss: 0.6367 - val_mean_squared_error: 0.6367
Epoch 5/5
2657/2657 [==============================] - 12s 5ms/step - loss: 2.1849 - mean_squared_error: 2.1849
| 102       | 2.185     | 0.8797    | 2.114     | 527.9     | 22.12     |


In [ ]:
best_param=bo.res[23]['params']
best_param

{'drop': 0.5233643407780469,
 'hidden_layers': 0.38648080556972186,
 'nb_batch': 793.7999516118144,
 'neurons': 22.435416267184642}

In [ ]:
reconstructed_model = keras.models.load_model("./model/0.9612308144569397.h5")

OSError: ignored

In [ ]:
Y_train_pred = reconstructed_model.predict(X_train)

In [ ]:
Y_test_pred = reconstructed_model.predict(X_test)

In [ ]:
print(X_test.shape,X_train.shape,y_test.shape,y_train.shape )

In [ ]:
y_test=y_test.reshape(-1,1)

In [ ]:
y_train=y_train.reshape(-1,1)

In [ ]:
import matplotlib.pyplot as plt
def evaluation(Y_real, Y_pred, graph_on=False):
    loss_length = len(Y_real.values.flatten()) - len(Y_pred)
    if loss_length != 0:
        Y_real = Y_real[loss_length:]
    if graph_on == True:
        pd.concat([Y_real, pd.DataFrame(Y_pred, index=Y_real.index, columns=['prediction'])], axis=1).plot(kind='line', figsize=(20,6),
                                                                                                           xlim=(Y_real.index.min(),Y_real.index.max()),
                                                                                                           linewidth=3, fontsize=20)
        plt.title('Time Series of Target', fontsize=20)
        plt.xlabel('Index', fontsize=15)
        plt.ylabel('Target Value', fontsize=15)
    MAE = abs(Y_real.values.flatten() - Y_pred).mean()
    MSE = ((Y_real.values.flatten() - Y_pred)**2).mean()
    MAPE = (abs(Y_real.values.flatten() - Y_pred)/Y_real.values.flatten()*100).mean()
    Score = pd.DataFrame([MAE, MSE, MAPE], index=['MAE', 'MSE', 'MAPE'], columns=['Score']).T
    Residual = pd.DataFrame(Y_real.values.flatten() - Y_pred, index=Y_real.index, columns=['Error'])
    return Score, Residual

def evaluation_trte(Y_real_tr, Y_pred_tr, Y_real_te, Y_pred_te, graph_on=False):
    Score_tr, Residual_tr = evaluation(Y_real_tr, Y_pred_tr, graph_on=graph_on)
    Score_te, Residual_te = evaluation(Y_real_te, Y_pred_te, graph_on=graph_on)
    Score_trte = pd.concat([Score_tr, Score_te], axis=0)
    Score_trte.index = ['Train', 'Test']
    return Score_trte, Residual_tr, Residual_te

In [ ]:
Score_GRU, Residual_tr, Residual_te = evaluation_trte(pd.DataFrame(y_train), Y_train_pred.flatten(), pd.DataFrame(y_test), Y_test_pred.flatten(), graph_on=True)

In [ ]:
display(Score_GRU)

In [ ]:
# nb_epoch=5
# neurons=3
# hidden_layers=3
# drop=0.5
# nb_batch=500

# # input layer
# inputs = tf.keras.Input(shape=(1,12))
# x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, recurrent_dropout=0.2, activation='relu')(inputs)  

# if hidden_layers !=0:
#     for i in range(1,hidden_layers+1):
#         x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)), return_sequences=True, activation='relu')(x)
# x = tf.keras.layers.GRU(neurons*(2**(hidden_layers+1)),return_sequences=False, activation='relu')(inputs)        
# x = tf.keras.layers.Dropout(drop)(x)


# # output
# predictions = Dense(1)(x)


# # modeling
# model = tf.keras.Model(inputs=inputs, outputs=predictions)
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode="min",patience=3,restore_best_weights=True, verbose=1)



# # learning
# model.fit(X_train, y_train, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping], validation_data=(X_valid, y_valid))


In [ ]:
# a,b=model.evaluate(X_test, y_test)

In [ ]:
# los_test